In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

import joblib

from data_preprocessing import pipeline

import os
import sys
from api import get_future_estate_list
from data_preprocessing_base import pipeline_base
from data_preprocessing_online import pipeline_online

### 데이터 전처리

In [8]:
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스.csv"
df = pd.read_csv(file_path, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_9748\43256658.py:2: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='cp949')


In [2]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스.csv"
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 전처리
preprocessing_pipeline =pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250317-01"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_9848\475702578.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='cp949')
c:\Users\user\developments\hk-project\src\data_preprocessing.py:136: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(


### 모델 학습 및 평가

In [7]:
# 전처리된 데이터 가져오기

file_version = "250317-01"
file_path = f"./storage/train_data/train-{file_version}.csv"
df = pd.read_csv(file_path, encoding='cp949')

In [8]:
df.columns

Index(['공고번호', '주택명', '공급지역코드', '공급지역명', '공급위치우편번호', '공급위치', '공급규모', '모집공고일',
       '청약접수시작일', '청약접수종료일', '당첨자발표일', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업',
       '공공주택지구', '대규모택지개발지구', '수도권내민영공공주택지구', '주택형', '공급세대수', '순위', '거주지역',
       '접수건수', '경쟁률', '최저당첨가점', '최고당첨가점', '평균당첨가점', '전용면적', '평면유형', '미달여부',
       '공급금액(최고가 기준)', '위도', '경도', '행정동코드', '법정동코드', '시도', '시군구', '읍면동1',
       '읍면동2', '전용면적당 공급금액(최고가기준)', '전용면적당 시세차익'],
      dtype='object')

In [6]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# model.fit(x, y)

### 모델 저장 

In [7]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

### 모델 예측

In [ ]:
# 모델 로드

loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

In [5]:
# 예측 데이터 로드
df_future_estate_list = get_future_estate_list()

In [6]:
# 예측 데이터 전처리

base_pipeline = pipeline_base()
online_pipeline = pipeline_online()

df_future_estate_list = base_pipeline.transform(df_future_estate_list)
df_future_estate_list = online_pipeline.transform(df_future_estate_list)

          공고번호               주택명 공급지역코드 공급지역명 공급위치우편번호  \
0   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
1   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
2   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
3   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
4   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
5   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
6   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
7   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
8   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
9   2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
10  2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   
11  2025000021  용현 우방 아이유쉘 센트럴마린    400    인천    22190   

                       공급위치  공급규모       모집공고일     청약접수시작일     청약접수종료일  ...  \
0   인천광역시 미추홀구 용현동 627-80번지   196  2025-03-14  2025-03-24  2025-03-26  ...   
1   인천광역시 미추홀구 용현동 627-80번지   196  2025-03-14  2025-03-24  2025-03-26  ...   
2   인천광역시 미

c:\Users\user\developments\hk-project\src\api.py:790: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0] # converters: 각 컬럼의 데이터를 문자열로 변환
c:\Users\user\developments\hk-project\src\api.py:704: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['정제된주택명'] = df['공급지역명'] + ' ' + df['주택명'].apply(clean_name)
c:\Users\user\developments\hk-project\src\api.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

1번 기사 - "41층에서 하버뷰를"용현 우방 아이유쉘 센트럴마린
2번 기사 - '안동용상하늘채리버스카이' 등 1205가구 분양예정[분양캘린더]
3번 기사 - 청년안심주택 보증금 238억 위험… 소형청약 경쟁률 160.9대 1 사상 최고...


In [4]:
# 전처리된 예측 데이터

df_future_estate_list

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,읍면동2,전용면적당 공급금액(최고가기준),전용면적당 시세차익,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7
0,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.747458e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
1,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.747458e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
2,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.747458e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
3,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.747458e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
4,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.626027e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
5,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.626027e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
6,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.626027e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
7,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.626027e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
8,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.632895e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767
9,2025000021,용현 우방 아이유쉘 센트럴마린,400,인천,22190,인천광역시 미추홀구 용현동 627-80번지,196,2025-03-14,2025-03-24,2025-03-26,...,용현동,6.632895e+06,NaN,0.179733,0.0011,0.0597,0.020833,0.081567,0.3104,0.346767


In [16]:
# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)